# Image Feature Extraction via PCA & kPCA

> *Numerical Optimization and Large Scale Linear Algebra*  
> *MSc in Data Science, Department of Informatics*  
> *Athens University of Economics and Business*

---

<p style='text-align: justify;'>For this task, we use images from the Yale Face Database B, which contains $5760$ single light source gray-level images of $10$ subjects, each seen under $576$ viewing conditions. We take $51$ images of the first subject and $51$ images of the third subject as the training data, and $13$ images of each of them as testing data. Then all the images are aligned, and each image has $168 \times 192$ pixels. We use the $168 \times 192$ pixel intensities as the original features for each image, thus the original feature vector is $32256$-dimensional. Then we use standard PCA and Gaussian kernel PCA to extract the $9$ most significant features from the training data, and record the eigenvectors. For standard PCA, only the eigenvectors are needed to extract features from testing data. For Gaussian kernel PCA, both the eigenvectors and the training data are needed to extract features from testing data. Note that for standard PCA, there are particular fast algorithms to compute the eigenvectors when the dimensionality is much higher than the number of data points. For kernel PCA, we use a Gaussian kernel with $\sigma = 22546$. For classification, we use the simplest linear classifier.</p>

<p style='text-align: justify;'>Parameter selection for kernel PCA directly determines the performance of the algorithm. For Gaussian kernel PCA, the most important parameter is the $\sigma$ in the kernel function defined by $\kappa(x,y)=\exp(-\|x-y\|^{2} \div 2\sigma^{2})$. The Gaussian kernel is a function of the distance $\| x-y \|$ between two vectors $x$ and $y$. Ideally, if we want to separate different classes in the new feature space, then the parameter $\sigma$ shoud be smaller than inter-class distances, and larger than inner-class distances. However, we don't know how many classes are there in the data, thus it is not easy to estimate the inter-class or inner class distances. Alternatively, we can set $\sigma$ to a small value to capture only the neighborhood information of each data point. For this purpose, for each data point $x_{i}$, let the distance from $x_{i}$ to its nearest neighbor be $d_{i}^{NN}$. In the experiments, we use this parameter selection strategy:<br><br>$$\sigma=5mean(d_{i}^{NN}).$$<br>This strategy, in the experiments, ensures that the $\sigma$ is large enough to capture neighboring data points, and is much smaller than inter-class distances. When using different datasets, this strategy may need modifications.</p>

> ***Kernel Principal Component Analysis and its Applications in Face Recognition and Active Shape Models***  
> *Quan Wang, Rensselaer Polytechnic Institute, 110 Eighth Street, Troy, NY 12180 USA*  
> *arXiv:1207.3538 \[cs.CV\], 2012*

## *Table of Contents*

- [*1. Libraries*](#libraries)
- [*2. Data*](#data)
- [*3. PCA*](#pca)
- [*4. Gaussian Kernel PCA*](#gaussian_kernel_pca)
    - [*4.1. Manual Implementation*](#manual)
    - [*4.2. Scikit Learn Implementation*](#sklearn)
- [*5. Classification Results*](#classification_results)

---

## Libraries <a class='anchor' id='libraries'></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import h5py
from sklearn.decomposition import PCA, KernelPCA
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import pdist, squareform

from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

## Data <a class='anchor' id='data'></a>

##### *Define a function to read the data from the `.mat` file*

In [2]:
# path and file of the data
filename = './data/YaleFaceData.mat'

def read_data(filename):
    
    # read .mat file
    f = h5py.File(filename, 'r')
    
    # define train and test set
    x_train, y_train = f['train_x'][:].T, f['train_t'][:]
    x_test, y_test = f['test_x'][:].T, f['test_t'][:]
    
    # shapes
    print(f'x_train.shape: {x_train.shape}')
    print(f'y_train.shape: {y_train.shape}')
    print(f' x_test.shape: {x_test.shape}')
    print(f' y_test.shape: {y_test.shape}')
    
    return x_train, y_train, x_test, y_test

# execute function
x_train, y_train, x_test, y_test = read_data(filename)

x_train.shape: (102, 32256)
y_train.shape: (102, 1)
 x_test.shape: (26, 32256)
 y_test.shape: (26, 1)


## PCA <a class='anchor' id='pca'></a>

<p style='text-align: justify;'>The <b>principal components</b> of a collection of points in a real coordinate space are a sequence of $p$ unit vectors, where the $i$-th vector is the direction of a line that best fits the data while being orthogonal to the first $i-1$ vectors. Here, a best-fitting line is defined as one that minimizes the average squared distance from the points to the line. These directions constitute an orthonormal basis in which different individual dimensions of the data are linearly uncorrelated. <b>Principal component analysis (PCA)</b> is the process of computing the principal components and using them to perform a change of basis on the data, sometimes using only the first few principal components and ignoring the rest.</p>

##### *Project train and test data onto a lower dimensional space*

In [3]:
def pca(x_train, x_test, n_components=9):
    """
    Principal Component Analysis
    ____________________________
    :param x_train: the input train data matrix, each row is one observation (M), each column is one feature (N)
    :param x_test: the input unseen data matrix, each row is one observation (M), each column is one feature (N)
    :param n_components: the number of principal components to keep
    :return x_train_PCA: the projection of the input train data onto a lower dimensional space
    :return x_test_PCA: the projection of the input unseen data onto a lower dimensional space
    ============================
    step1: initialize PCA instance
    step2: fit PCA instance into the training set
    step3: get the 9 largest eigenvectors as principal components
    step3: project data onto a lower dimensional space
    """
    
    # initialize PCA instance
    pca = PCA(n_components=n_components, random_state=1)
    
    # fit PCA instance
    # into x_train
    pca.fit(x_train)
    
    # get the 9 largest eigenvectors
    # as principal components
    eigenvectors = pca.components_
    
    # project data onto a lower dimensional space
    x_train_PCA = np.dot(x_train, eigenvectors.T)
    x_test_PCA = np.dot(x_test, eigenvectors.T)
    
    return x_train_PCA, x_test_PCA

# execute function
x_train_PCA, x_test_PCA = pca(x_train, x_test)

## Gaussian Kernel PCA <a class='anchor' id='gaussian_kernel_pca'></a>

<p style='text-align: justify;'>In the field of multivariate statistics, <b>kernel principal components analysis (kernel PCA)</b> is an extension of PCA that allows for the separability of non-linear data by making use of kernels. The basic idea behind it is to project the linearly inseparable data onto a higher dimensional space where it becomes linearly separable.</p>

### *Manual Implementation* <a class='anchor' id='manual'></a>

##### *Steps*

1. Compute hyperparameter $\sigma$
2. Use Gaussian kernel PCA to project the train data onto a lower dimensional space and record the eigenvectors
3. Use Gaussian kernel PCA and the eigenvectors obtained from previous step, to project the test data onto a lower dimensional space

##### *Compute hyperparameter $\sigma$*

In [4]:
def compute_sigma(x):
    """
    Hyperparameter σ
    ________________
    :param x: the input data matrix, each row is one observation (M), each column is one feature (N)
    :return: the σ parameter
    ================
    step1: calculate the pairwise distances between all data points of the input matrix
    step2: find the distance, be it di^NN, of each data point xi from its nearest neighbor
    step3: calculate the mean of the smallest distances
    step4: compute σ, σ = 5 x mean(di^NN)
    """
    
    # calculate pairwise distances
    distances = pairwise_distances(x)
    distances = np.where(distances == 0, np.inf, distances)
    
    # find the distance of each data point from its nearest neighbor
    min_distances = [min(dist) for dist in distances]
    
    # calculate the avg smallest distance
    mean_distance = np.mean(min_distances)
    
    # compute sigma
    sigma = 5 * mean_distance
    
    return sigma, print(f'σ = {int(sigma)}')

# execute function
sigma, _ = compute_sigma(x_train)

σ = 22546


##### *Use Gaussian kernel PCA to project the train data onto a lower dimensional space and record the eigenvectors*

In [5]:
def kPCA(x, sigma, n_components=9):
    """
    Kernel PCA algorithm
    ____________________
    :param x: the input data matrix, each row is one observation (M), each column is one feature (N)
    :param sigma: the hyperparameter σ
    :param n_components: the number of principal components to keep
    :return x_train_kPCA: the projection of the input data onto a lower dimensional space
    :return eigenvectors: the principal [n_components] components
    ====================
    step1: construct the kernel matrix K0
    step2: compute the gram matrix (aka center the kernel matrix) $K=K0-1_NK0-K01_N+1_NK01_N$
    step3: compute the eigenvalues and the eigenvectors of $K^C$
    step4: normalize the eigenvectors
    step5: get the k first largest eigenvectors to become principal components
    step6: project the input data onto a lower dimensional space
    """
    
    def rbf_kernel(x, sigma):
        """
        Gaussian rbf kernel
        ___________________
        :param x: the input data matrix, each row is one observation (M), each column is one feature (N)
        :param sigma: the hyperparameter σ
        :return: the rbf kernel, $e^{-||x-y||^2/2*sigma^2}$
        ===================
        step1: calculate the squared euclidean distance for every pair of points in the MxN dimensional dataset
        step2: convert the pairwise distances into a symmetric MxM matrix
        step3: compute the MxM kernel matrix
        """
        D = pdist(x,'sqeuclidean')
        D = squareform(D)
        D = np.where(D<0,0,D)
        K = np.sqrt(D)
        K = K**2
        K = np.exp(-K/(2*sigma**2))
        return K
    
    # construct the kernel matrix
    K0 = rbf_kernel(x, sigma)
    
    # compute the gram matrix
    N = x.shape[0]
    oneN = np.ones((N,N))/N
    K = K0 - np.dot(oneN,K0) - np.dot(K0,oneN) + np.dot(np.dot(oneN,K0),oneN)
    
    # eigenvalue analysis
    eigenvalues, eigenvectors = np.linalg.eigh(K)
    
    # normalization
    norm_eigenvectors = np.sqrt(sum(eigenvectors**2))
    eigenvectors = eigenvectors / np.tile(norm_eigenvectors, (eigenvectors.shape[0], 1))
    
    # dimensionality reduction
    kLargestIdx = np.argsort(eigenvalues)[::-1][:n_components]
    eigenvectors = eigenvectors[:,kLargestIdx]
    x_train_kPCA = np.dot(K0,eigenvectors)
    
    return x_train_kPCA, eigenvectors

# execute function
x_train_kPCA, eigenvectors = kPCA(x_train, sigma)

##### *Use Gaussian kernel PCA and the eigenvectors obtained from previous step, to project the test data onto a lower dimensional space*

In [6]:
def kPCA_newData(x_train, x_test, eigenvectors, sigma):
    """
    Kernel PCA algorithm for unseen data
    ____________________________________
    :param x_train: the input train data matrix, each row is one observation (M), each column is one feature (N)
    :param x_test: the input unseen data matrix, each row is one observation (M), each column is one feature (N)
    :param eigVector: the eigenvectors obtained from the previous step
    :param sigma: the hyperparameter σ
    :return x_test_kPCA: the projection of the input unseen data onto a lower dimensional space
    ====================================
    step1: construct the kernel matrix K 
    step2: compute the projection of the input data onto a lower dimensional space
    """
    
    def rbf_kernel_newData(x_train, x_test, sigma):
        """
        Gaussian rbf kernel
        ___________________
        :param x_train: the input train data matrix, each row is one observation (M), each column is one feature (N)
        :param x_test: the input unseen data matrix, each row is one observation (M), each column is one feature (N)
        :param sigma: the hyperparameter σ
        :return: the rbf kernel, $e^{-||x-y||^2/2*sigma^2}$
        ===================
        step1: calculate the squared euclidean distance for every pair of points in the MxN dimensional dataset
        step2: convert the pairwise distances into a symmetric MxM matrix
        step3: compute the MxM kernel matrix
        """
        D = pdist(np.concatenate([x_train,x_test]), 'sqeuclidean')
        D = squareform(D)
        D = np.where(D<0,0,D)
        K = np.sqrt(D)
        N = x_train.shape[0]
        K = K[N:,:N]
        K = K**2
        K = np.exp(-K/(2*sigma**2))
        return K
    
    # construct the kernel matrix
    K = rbf_kernel_newData(x_train, x_test, sigma)
    
    # dimensionality reduction
    x_test_kPCA = np.dot(K,eigenvectors)
    
    return x_test_kPCA

# execute function
x_test_kPCA = kPCA_newData(x_train, x_test, eigenvectors, sigma)

## *Scikit Learn Implementation*

##### *Use the scikit-learn kernel PCA algorithm to project train and test data onto a lower dimensional space*

In [7]:
def kPCA_sklearn(x_train, x_test, sigma, n_components=9):
    """
    Kernel PCA algorithm using scikit-learn implementation
    ______________________________________________________
    :param x_train: the input train data matrix, each row is one observation (M), each column is one feature (N)
    :param x_test: the input unseen data matrix, each row is one observation (M), each column is one feature (N)
    :param sigma: the hyperparameter σ
    :param n_components: the number of principal components to keep
    :return x_train_PCA_2: the projection of the input train data onto a lower dimensional space
    :return x_test_PCA_2: the projection of the input unseen data onto a lower dimensional space
    ======================================================
    step1: initialize kernel PCA instance
    step2: fit and project both input and unseen data onto a lower dimensional space
    """

    # initialize kernel PCA instance
    kPCA = KernelPCA(n_components=n_components, kernel='rbf', gamma=(1/(2*sigma**2)))

    # project train and test data in a lower dimension
    x_train_kPCA_2 = kPCA.fit_transform(x_train)
    x_test_kPCA_2 = kPCA.transform(x_test)
    
    return x_train_kPCA_2, x_test_kPCA_2

# execute function
x_train_kPCA_2, x_test_kPCA_2 = kPCA_sklearn(x_train, x_test, sigma)

## Classification Results <a class='anchor' id='classification_results'></a>

##### *Comparing classification results between the two dimensionality reduction methods*

In [8]:
def compute_error_rates(x_fit, y_fit, x_pred, y_true):
    # initialize model
    model = LinearRegression()
    # fit the model in the data
    model.fit(x_fit, y_fit.ravel())
    # make predictions
    predictions = model.predict(x_pred)
    # translate prediction
    predictions = np.where(predictions>0,1,-1)
    # compute error rate
    error_rate = 1 - accuracy_score(y_true, predictions)
    return error_rate

# PCA train
error_rate = compute_error_rates(x_train_PCA, y_train, x_train_PCA, y_train)
print(f' PCA train error: {round(error_rate*100,2)}%')

# PCA test
error_rate = compute_error_rates(x_train_PCA, y_train, x_test_PCA, y_test)
print(f'  PCA test error: {round(error_rate*100,2)}%\n')

# Kernel PCA train (manual)
error_rate = compute_error_rates(x_train_kPCA, y_train, x_train_kPCA, y_train)
print(f'kPCA train error: {round(error_rate*100,2)}% - (manual)')

# Kernel PCA test (manual)
error_rate = compute_error_rates(x_train_kPCA, y_train, x_test_kPCA, y_test)
print(f' kPCA test error: {round(error_rate*100,2)}% - (manual)\n')

# Kernel PCA train (sklearn)
error_rate = compute_error_rates(x_train_kPCA_2, y_train, x_train_kPCA_2, y_train)
print(f'kPCA train error: {round(error_rate*100,2)}% - (sklearn)')

# Kernel PCA test (sklearn)
error_rate = compute_error_rates(x_train_kPCA_2, y_train, x_test_kPCA_2, y_test)
print(f' kPCA test error: {round(error_rate*100,2)}% - (sklearn)')

 PCA train error: 8.82%
  PCA test error: 23.08%

kPCA train error: 6.86% - (manual)
 kPCA test error: 11.54% - (manual)

kPCA train error: 6.86% - (sklearn)
 kPCA test error: 11.54% - (sklearn)


---

*Thank you!*

---